In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "64"

In [ ]:
import healpy as hp
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from pathlib import Path
from astropy.tests.helper import assert_quantity_allclose
import pickle
import sys
import os
from astropy import units as u

In [ ]:
from astropy.table import QTable

In [ ]:
cd ..

In [ ]:
instrument_model = QTable.read("instrument_model/cmbs4_instrument_model.tbl", format="ascii.ipac")
instrument_model.add_index("band")

In [ ]:
instrument_model.columns

In [ ]:
instrument_model.loc["LFL1"]

In [ ]:
dataset = "noise_atmo_7splits"

In [ ]:
ch = f"LAT-MFPL1_pole"
nsplits = 1

In [ ]:
#if os.path.exists(f"plots/C_ell_{dataset}_{ch}_{nsplits}.png"):
#    sys.exit(0)

In [ ]:
sampling_frequency = dict(SAT=100*u.Hz, LAT=400*u.Hz)

In [ ]:
telescope = ch.split("-")[0]
channel, site= ch.split("-")[1].split("_")

In [ ]:
local_path = Path("output")
project_path = Path("output/")

In [ ]:
C_ell = pickle.load(open(local_path/ dataset / f"C_ell_{telescope}_{site}_{nsplits}.pkl", "rb"))

In [ ]:
telescope, channel, site

In [ ]:
C_ell.keys()

In [ ]:
nside = 4096 if telescope == "LAT" else 512

In [ ]:
from s4_design_sim_tool.cli import parse_config
config = parse_config("./s4_baseline_design.toml", f"{dataset}.toml")

In [ ]:
hitmap = hp.read_map(project_path / dataset / ch / f"cmbs4_hitmap_{ch}_nside{nside}_1_of_{nsplits}.fits")

In [ ]:
ls output/noise_atmo_7splits/SA

In [ ]:
from s4_design_sim_tool.noise import get_tube_years
tube_years = get_tube_years(config, site, channel)
tube_years

In [ ]:
config["experiment"]["observing_efficiency"][site.capitalize()]

In [ ]:
config["experiment"]["sensitivity_factor"][site.capitalize()]

In [ ]:
sampling_rate = hitmap.sum()
sampling_rate /= 365.25 * 24 * 3600
sampling_rate /= tube_years * \
    config["experiment"]["observing_efficiency"][site.capitalize()][telescope][channel] * \
    config["experiment"]["sensitivity_factor"][site.capitalize()][telescope].get(channel, \
    config["experiment"]["sensitivity_factor"][site.capitalize()][telescope]["default"])
sampling_rate /= instrument_model.loc[channel]["detectors_per_tube"]
sampling_rate *= nsplits
sampling_rate

In [ ]:
np.testing.assert_allclose(sampling_rate, sampling_frequency[telescope].value, rtol=1/100)

In [ ]:
log_hitmap = np.log10(hitmap)
log_hitmap[np.isinf(log_hitmap)] = hp.UNSEEN
hp.mollview(log_hitmap, unit="log10(samples)", title=f"Hitmap {ch}")
plt.savefig(f"plots/hitmap_{channel}_{site}_1_of_{nsplits}.png")

In [ ]:
pixarea = hp.nside2pixarea(nside)
sky_fraction = (hitmap>0).sum()

In [ ]:
wcov_filename = project_path / dataset / ch / f"cmbs4_wcov_{ch}_nside{nside}_1_of_{nsplits}.fits"

In [ ]:
from compute_cl_utils import build_inverse_cov_weights

In [ ]:
temp_weights, pol_weights, noise_cl = build_inverse_cov_weights(wcov_filename)

In [ ]:
noise_cl = noise_cl[[True,True,False,True]]

In [ ]:
import astropy.units as u

In [ ]:
noise_cl

In [ ]:
wcov_from_hitmap = instrument_model.loc[channel]["NET"]**2 / (hitmap / sampling_frequency[telescope])

In [ ]:
wcov_from_hitmap[np.isinf(wcov_from_hitmap)] = 0

In [ ]:
white_noise_Cl = (np.average(wcov_from_hitmap, weights=temp_weights**2) * pixarea ).to(u.K**2)

In [ ]:
temp_weights

In [ ]:
white_noise_Cl

In [ ]:
del hitmap, log_hitmap

## Atmosphere

In [ ]:
from s4_design_sim_tool.atmosphere import load_atmosphere

In [ ]:
input_atmophere = hp.ma(load_atmosphere(config, site, channel, raw=True))

In [ ]:
input_atmophere[1:] *= pol_weights

In [ ]:
input_atmophere[0] *= temp_weights

In [ ]:
from s4_design_sim_tool.atmosphere import get_telecope_years

In [ ]:
from s4_design_sim_tool.core import simulations_observing_efficiency

In [ ]:
telescope_years = get_telecope_years(config, site, channel)

In [ ]:
atmo_cl = \
hp.anafast(input_atmophere, use_pixel_weights=True) 
atmo_cl[0] /= np.mean(temp_weights**2)
atmo_cl[1:] /= np.mean(pol_weights**2) # anyway we don't use TE TB

In [ ]:
atmo_cl *= 10 / 365.25
atmo_cl *= simulations_observing_efficiency[site.lower()][channel] / config["experiment"]["observing_efficiency"][site.capitalize()][telescope][channel]
#atmo_cl *= 1 / config["experiment"]["observing_efficiency"][site.capitalize()][telescope][channel]
atmo_cl /= telescope_years

In [ ]:
del input_atmophere

## Generate summary plot

In [ ]:
plt.style.use("seaborn-poster")

In [ ]:
instrument_model.loc[channel]

In [ ]:
for pols in [[(0, "TT")] , [(1, "EE"), (2, "BB")]]:
    label = "temp" if pols[0][1] == "TT" else "pol"
    plt.figure()
    
    try:
        noise_dsr = ((instrument_model.loc[channel][f"{site}_{label}_sensitivity_DSR"])**2).to_value(u.K**2 * u.radian**2)
    except KeyError as e:
        print("Missing value", e)
        noise_dsr = None
    
    for i, pol in pols:
        for split in range(1, nsplits+1):
            high_ell = C_ell[ch][split][i][1000:].mean()
            compare_wcov =  high_ell / noise_cl[i] * 100 - 100
            compare_dsr = 0 if noise_dsr is None else (high_ell / noise_dsr * 100 - 100)
            compare = "" if compare_wcov > 1000 else f"({compare_wcov:+.1f}% of wcov, {compare_dsr:+.1f}% of DSR)"
            plt.loglog(C_ell[ch][split][i], alpha=.8, label=pol + compare)

    for i, pol in pols:
        plt.loglog(atmo_cl[i]*nsplits, alpha=.8, ls="--", label=f"atmo {pol}")

    for i, pol in pols:
        plt.axhline(noise_cl[i], alpha=.8, ls="-.", color="gray", label=f"noise from wcov {pol}")
    
    plt.axhline((white_noise_Cl*2).to_value(u.K**2) if label == "pol" else white_noise_Cl.to_value(u.K**2), alpha=.8, ls="-.", color="yellow", label=f"noise from hitmap and NET")

    if noise_dsr is not None:
        plt.axhline(noise_dsr, ls=":",
           label=f"white noise {label} from DSR", color="purple", alpha=.5)

    plt.legend()
    plt.title(f"Noise + Atmosphere simulations spectra for {ch}, {label}")
    plt.ylabel("$C_\ell [K^2]$")
    plt.xlabel("$\ell$")
    plt.xlim((10, 3*nside))
    plt.grid(True)
    plt.savefig(f"plots/C_ell_{dataset}_{ch}_{nsplits}_{label}.png");